In [1]:
import importlib

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

%matplotlib inline

In [2]:
model_module = importlib.import_module("handyrl.model")
env_module = importlib.import_module("handyrl.envs.kaggle.hungry_geese")

Loading environment football failed: No module named 'gfootball'


In [3]:
e = env_module.Environment()
e.reset()

In [4]:
obs = e.observation()
obs = torch.from_numpy(obs.reshape(1, obs.shape[0], obs.shape[1], obs.shape[2])).clone()
obs_ = torch.cat([obs, obs], dim=0)
print(f"size: {obs_.size()}, type: {obs_.dtype}")

size: torch.Size([2, 17, 7, 11]), type: torch.float32


In [5]:
input_ = torch.randn(2, obs.shape[1], obs.shape[2], obs.shape[3])
print(f"size: {input_.size()}, type: {input_.dtype}")

size: torch.Size([2, 17, 7, 11]), type: torch.float32


In [6]:
net = env_module.GeeseNet()

In [7]:
# Disable trainable for pretrained weights.
# for param in net.offensive.parameters():
#     param.requires_grad = False
# for param in net.defensive.parameters():
#     param.requires_grad = False

In [8]:
out = net(obs_)
# print(f"size: {out.size()}, type: {out.dtype}")
print(f"size: {out['policy'].size()}, type: {out['policy'].dtype}")
print(f"size: {out['value'].size()}, type: {out['value'].dtype}")

size: torch.Size([2, 4]), type: torch.float32
size: torch.Size([2, 1]), type: torch.float32


In [9]:
out

{'policy': tensor([[-0.7532,  0.8419,  0.2019,  0.5431],
         [-0.7532,  0.8419,  0.2019,  0.5431]], grad_fn=<MmBackward>),
 'value': tensor([[0.0632],
         [0.0632]], grad_fn=<TanhBackward>),
 'h_head_p': tensor([[0.3135, 0.0000, 0.0000, 0.0000, 0.0000, 2.2778, 0.0000, 0.0000, 0.1175,
          0.5664, 0.0000, 0.4821, 1.6687, 0.0000, 2.0738, 0.0000, 0.3225, 0.0000,
          0.0000, 2.9552, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          1.1872, 1.1098, 0.0000, 0.0413, 0.0000],
         [0.3135, 0.0000, 0.0000, 0.0000, 0.0000, 2.2778, 0.0000, 0.0000, 0.1175,
          0.5664, 0.0000, 0.4821, 1.6687, 0.0000, 2.0738, 0.0000, 0.3225, 0.0000,
          0.0000, 2.9552, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          1.1872, 1.1098, 0.0000, 0.0413, 0.0000]], grad_fn=<SumBackward1>),
 'h_head_v': tensor([[0.4009, 0.4819, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.0593,
          0.5346, 0.0000, 0.0000, 0.0000, 1.0529, 0.0000, 0.0000, 1.4498,

In [10]:
# pytorch_total_params
params = sum(p.numel() for p in net.parameters())
print(f"{params:,}")

137,568


In [11]:
# pytorch_total_params (trainable)
params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f"{params:,}")

137,568


In [12]:
net

GeeseNet(
  (conv0): TorusConv2d(
    (conv): Conv2d(17, 32, kernel_size=(3, 3), stride=(1, 1))
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (blocks): ModuleList(
    (0): TorusConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): TorusConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): TorusConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): TorusConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): TorusConv2d(
      (conv): Conv2d(32, 32,